<h1 style="font-size: 50px;">Research Project 2 Epilepsy Ionic Modulation - SmallBrain Cortex Layers</h1>

This file contains the code for running a simulation where the model used is build by mimicking the layers present in the neocortex: L1, L2/L3, L4, L5, and L6. For each layer, the neuron densities and volumes were retrieved from literature and were used to generate ~17000 neurons across the entire 3D space.

<br></br><br></br>

<h2 style="font-size: 40px;">Installing and Importing Libraries</h2>

In [1]:
# Installing all the required libraries.
!pip install -q jupyter
!pip install -q matplotlib
!pip install -q numpy
!pip install -q pandas
!pip install -q plotly
!pip install -q scipy
!pip install -q tqdm
!pip install -q Brian2

In [2]:
# Importing all the required libraries.
from plots import *
from equations import *
from global_settings import *
from masks import *
from helper import *
from run_loop import *

In [3]:
import os
from brian2 import *
import random
from itertools import chain, zip_longest
from tqdm.notebook import tqdm
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

<br></br><br></br>

<h2 style="font-size: 40px;">General Functions</h2>

In [4]:
# Reads the stimulus input signal.
def read_input_signal(file_name):
    in_1 = np.loadtxt('./stimuli/'+file_name)

    # Converting the signal to be of type 'TimedArray' with a specified time step which can be used in the simulation.
    input_signal = TimedArray(in_1*namp,dt=defaultclock.dt)

    return input_signal

<br></br>

In [5]:
# Checking whether the mask with the provided coordinate center and radius can be applied to the list of layers.
def checking_mask(shape_mask, list_of_layers, center_coordinates, radius_or_edge_length, layer_names, layer_geometrics):

    # Checking if the 'list_of_layers' is empty.
    if len(list_of_layers) == 0:
        raise ValueError("The 'list_of_layers' list is empty.")
    
    # Checking if the 'list_of_layers' only contains valid layer names.
    if not all(layer in layer_names for layer in list_of_layers):
        raise ValueError("The 'list_of_layers' list contains an invalid layer name.")
    
    # Checking if there are no duplicate layer in the 'list_of_layers'.
    if len(list_of_layers) != len(set(list_of_layers)):
        raise ValueError("The 'list_of_layers' list contains duplicate layers.")

    # Checking that the 'list_of_layers' list only contains layers that are adjacent to each other (otherwise the stimulus mask would pass through layers that are not in this list).
    if len(list_of_layers) > 1:
        layer_indices = [layer_names.index(layer) for layer in list_of_layers]
        if not all(layer_indices[i] + 1 == layer_indices[i+1] for i in range(len(layer_indices) - 1)):
            raise ValueError("The 'list_of_layers' list contains layers that are not adjacent to each other.")

    # Checking if the coordinates of the center of the mask are positive.
    if not (center_coordinates[0] > 0 and center_coordinates[1] > 0 and center_coordinates[2] > 0):
        raise ValueError("The coordinates of the center of the mask should be positive.")

    # Checking if the radius of the mask is positive.
    if not (radius_or_edge_length > 0):
        raise ValueError("The radius of the mask should be positive.")

    # Generating 'num_edge_points' random points that are located on the edge of the mask which is either spherical, cubical, or all.
    num_edge_points = 10000
    if shape_mask == "spherical":
        radius = radius_or_edge_length
        edge_points = generating_n_random_points_spherical_edge(num_edge_points, center_coordinates, radius)
        
    elif shape_mask == "cubical":
        edge_length = radius_or_edge_length
        edge_points = generating_n_random_points_cubical_edge(num_edge_points, center_coordinates, edge_length)
        
    else:
        raise ValueError("The shape of the mask given is not valid. Please select either 'spherical' or 'cubical'.")

    # Checking whether every point present in the 'edge_points' is located within the list of layers.
    for edge_point in edge_points:
        point_in_layer = False
        for layer in list_of_layers:
            match layer:
                case "L1":
                    point_in_layer = check_point_within_layer(edge_point, 'L1', layer_geometrics)
                case "L2_L3":
                    point_in_layer = check_point_within_layer(edge_point, 'L2_L3', layer_geometrics)
                case "L4":
                    point_in_layer = check_point_within_layer(edge_point, 'L4', layer_geometrics)
                case "L5":
                    point_in_layer = check_point_within_layer(edge_point, 'L5', layer_geometrics)
                case "L6":
                    point_in_layer = check_point_within_layer(edge_point, 'L6', layer_geometrics)
            if point_in_layer:
                break
        if not point_in_layer:
            raise ValueError("Not every point present in the mask will be within the defined layers.")

    return [center_coordinates, radius_or_edge_length]

<br></br>

In [6]:
# Generating n random points that are located on the edge of the spherical mask.
def generating_n_random_points_spherical_edge(num_points, center_coordinates, radius):

    # Looping 'num_points' times and for each loop generating a random point on the edge of the spherical mask.
    points = []
    for i in range(num_points):
        # Generating the components needed to generate a point on the edge exactly 'radius' away from the center coordinates.
        phi = np.random.uniform(0, 2 * np.pi)
        costheta = np.random.uniform(-1, 1)
        theta = np.arccos(costheta)

        # Generating a random point that is located on the edge of the spherical mask.
        x = center_coordinates[0] + radius * np.sin(theta) * np.cos(phi)
        y = center_coordinates[1] + radius * np.sin(theta) * np.sin(phi)
        z = center_coordinates[2] + radius * np.cos(theta)

        # Appending the randomly generated point to the 'points' array.
        points.append([x, y, z])
        
    return points

<br></br>

In [7]:
# Generating n random points that are located on the edge of the cubical mask.
def generating_n_random_points_cubical_edge(num_points, center_coordinates, edge_length):

    # Looping 'num_points' times and for each loop generating a random point on the edge of the cubical mask.
    points = []
    for i in range(num_points):
        
        # Generating a random face of the cube on which the random point will be located.
        face = np.random.randint(0, 6)  

        # Depending on the face generated, a random point will be created.
        match face:
        
            # The right hand side face.
            case 0: 
                x = center_coordinates[0] - edge_length/2
                y = np.random.uniform(center_coordinates[1] - edge_length/2, center_coordinates[1] + edge_length/2)
                z = np.random.uniform(center_coordinates[2] - edge_length/2, center_coordinates[2] + edge_length/2)

            # The left hand side face.
            case 1:
                x = center_coordinates[0] + edge_length/2
                y = np.random.uniform(center_coordinates[1] - edge_length/2, center_coordinates[1] + edge_length/2)
                z = np.random.uniform(center_coordinates[2] - edge_length/2, center_coordinates[2] + edge_length/2)

            # The bottom face.
            case 2:
                x = np.random.uniform(center_coordinates[0] - edge_length/2, center_coordinates[0] + edge_length/2)
                y = center_coordinates[1] - edge_length/2
                z = np.random.uniform(center_coordinates[2] - edge_length/2, center_coordinates[2] + edge_length/2)

            # The top face.
            case 3:
                x = np.random.uniform(center_coordinates[0] - edge_length/2, center_coordinates[0] + edge_length/2)
                y = center_coordinates[1] + edge_length/2
                z = np.random.uniform(center_coordinates[2] - edge_length/2, center_coordinates[2] + edge_length/2)

            # The front face.
            case 4:
                x = np.random.uniform(center_coordinates[0] - edge_length/2, center_coordinates[0] + edge_length/2)
                y = np.random.uniform(center_coordinates[1] - edge_length/2, center_coordinates[1] + edge_length/2)
                z = center_coordinates[2] - edge_length/2

            # The rear face.
            case 5:
                x = np.random.uniform(center_coordinates[0] - edge_length/2, center_coordinates[0] + edge_length/2)
                y = np.random.uniform(center_coordinates[1] - edge_length/2, center_coordinates[1] + edge_length/2)
                z = center_coordinates[2] + edge_length/2

        # Appending the randomly generated point to the 'points' array.
        points.append([x, y, z])

    return points

<br></br>

In [8]:
# Checking whether the point is located within the current layer.
def check_point_within_layer(point, layer, layer_geometrics):

    # Retrieving the x-coordinate, y-coordinate, and z-coordinate from the 'point' parameter.
    x_point, y_point, z_point = point

    # Determining for which layer the point should be checked.
    match layer:
        case "L1":
            layer_block = layer_geometrics['L1_block']
        case "L2_L3":
            layer_block = layer_geometrics['L2_L3_block']
        case "L4":
            layer_block = layer_geometrics['L4_block']
        case "L5":
            layer_block = layer_geometrics['L5_block']
        case "L6":
            layer_block = layer_geometrics['L6_block']

    # Retrieving the bottom left x-coordinate, y-coordinate, and z-coordinate of the current layer from the 'layer_geometrics' parameter.
    layer_bottom_left_x, layer_bottom_left_y, layer_bottom_left_z  = layer_block['bottom_left_position']

    # Retrieving the width, height, and width of the current layer from the 'layer_geometrics' parameter.
    layer_width = layer_block['width']
    layer_height = layer_block['height']
    layer_depth = layer_block['depth']

    # Checking whether the point is located within the current layer.
    if ((x_point >= layer_bottom_left_x and x_point <= layer_bottom_left_x + layer_width) and
        (y_point >= layer_bottom_left_y and y_point <= layer_bottom_left_y + layer_height) and
        (z_point >= layer_bottom_left_z and z_point <= layer_bottom_left_z + layer_depth)):
        return True
    else:
        return False

<br></br><br></br>

<h2 style="font-size: 40px;">Topology and Neuron Groups Functions</h2>

In [9]:
# Creating the complete neuron topology of the model taking into account the structure of the cortex.
def create_complete_neuron_topology_cortex(desired_total_num_of_neurons, layer_neuron_densities, layer_volumes, layer_excitatory_ratios, layer_names):

    ###########################################################
    ########  Calculating Number of Neurons per layer  ########
    ###########################################################
    # Initializing a dictionary that will contain the number of neurons present in each of the layers and a variable that counts the total numbers of neurons.
    num_of_neurons = {}
    total_num_of_neurons = 0

    # Looping over every layer and calculating how many neurons are present in each of the layers depending on the neuron densities and the volumes of the layers.
    for layer in layer_names:
        num_of_neurons[layer] = layer_neuron_densities[layer] * layer_volumes[layer]
        total_num_of_neurons += num_of_neurons[layer]

    # Calculating the scaling factor for the number of neurons for all layers as we ideally only want to model a number similar to the 'desired_total_num_of_neurons'.
    neuron_scaling_factor = desired_total_num_of_neurons / total_num_of_neurons

    # Initializing a dictionary that will contain the adjusted number of neurons present in each of the layers and one that will contain the adjusted volume of each layer.
    num_of_neurons_adjusted = {}
    layer_volume_adjusted = {}

    # Looping over every layer and calculating how many neurons are present in each of the layers after having multiplied the original number with the 'neuron_scaling_factor'.
    for layer in layer_names:
        num_of_neurons_adjusted[layer] = neuron_scaling_factor * num_of_neurons[layer]
        layer_volume_adjusted[layer] = neuron_scaling_factor * layer_volumes[layer]

    # Creating a dictionary that will eventually contain all the layer information.
    layer_data = {}
    for layer in layer_names:
        layer_data[layer] = {"layer_name": layer,
                             "volume": layer_volume_adjusted[layer],
                             "num_excitatory_neurons": int(layer_excitatory_ratios[layer] * num_of_neurons_adjusted[layer]),
                             "num_inhibitory_neurons": int((1 - layer_excitatory_ratios[layer]) * num_of_neurons_adjusted[layer])}

    # Initializing a dictionary that will eventually contain the geometrical information of the different building blocks of the layers.
    layer_geometrics = {}

    
    ##############################################################
    ########  Calculating Width and Depth of Every Layer  ########
    ##############################################################
    # Calculating the width and depth of every layer (which is the same for every layer) by focusing on 1 layer and defining its width:height:depth ratio. For this we will use layer L1 and and use a ratio of 1:0.5:1.
    width_layers = (2 * layer_data['L1']['volume']) ** (1/3)
    depth_layers = width_layers

    
    #####################################################
    ########  Calculating Height of Every Layer  ########
    #####################################################
    # Calculating the height of every layer by dividing the volume of each layer by the product of the calculated constant width and depth.
    L1_height = layer_data['L1']['volume'] / (width_layers * depth_layers)
    L2_L3_height = layer_data['L2_L3']['volume'] / (width_layers * depth_layers)
    L4_height = layer_data['L4']['volume'] / (width_layers * depth_layers)
    L5_height = layer_data['L5']['volume'] / (width_layers * depth_layers)
    L6_height = layer_data['L6']['volume'] / (width_layers * depth_layers)


    #####################################################
    ########  Geometric Information of Layer L6  ########
    #####################################################
    # Adding the geometrical information of layer L6 to the 'layer_geometrics' database.
    L6_x = 0
    L6_y = 0
    L6_z = 0
    layer_geometrics['L6_block'] = {"width": width_layers,
                                    "height": L6_height,
                                    "depth": depth_layers,
                                    "bottom_left_position": [L6_x, L6_y, L6_z],
                                    "num_excitatory_neurons": layer_data['L1']['num_excitatory_neurons'],
                                    "num_inhibitory_neurons": layer_data['L1']['num_inhibitory_neurons']
                                    }


    #####################################################
    ########  Geometric Information of Layer L5  ########
    #####################################################
    # Adding the geometrical information of layer L5 to the 'layer_geometrics' database.
    L5_x = 0
    L5_y = L6_height
    L5_z = 0
    layer_geometrics['L5_block'] = {"width": width_layers,
                                    "height": L5_height,
                                    "depth": depth_layers,
                                    "bottom_left_position": [L5_x, L5_y, L5_z],
                                    "num_excitatory_neurons": layer_data['L1']['num_excitatory_neurons'],
                                    "num_inhibitory_neurons": layer_data['L1']['num_inhibitory_neurons']
                                    }


    #####################################################
    ########  Geometric Information of Layer L4  ########
    #####################################################
    # Adding the geometrical information of layer L4 to the 'layer_geometrics' database.
    L4_x = 0
    L4_y = L5_y + L5_height
    L4_z = 0
    layer_geometrics['L4_block'] = {"width": width_layers,
                                    "height": L4_height,
                                    "depth": depth_layers,
                                    "bottom_left_position": [L4_x, L4_y, L4_z],
                                    "num_excitatory_neurons": layer_data['L1']['num_excitatory_neurons'],
                                    "num_inhibitory_neurons": layer_data['L1']['num_inhibitory_neurons']
                                    }
    
    
    ########################################################
    ########  Geometric Information of Layer L2_L3  ########
    ########################################################
    # Adding the geometrical information of layer L2_L3 to the 'layer_geometrics' database.
    L2_L3_x = 0
    L2_L3_y = L4_y + L4_height
    L2_L3_z = 0
    layer_geometrics['L2_L3_block'] = {"width": width_layers,
                                    "height": L2_L3_height,
                                    "depth": depth_layers,
                                    "bottom_left_position": [L2_L3_x, L2_L3_y, L2_L3_z],
                                    "num_excitatory_neurons": layer_data['L1']['num_excitatory_neurons'],
                                    "num_inhibitory_neurons": layer_data['L1']['num_inhibitory_neurons']
                                    }

    
    #####################################################
    ########  Geometric Information of Layer L1  ########
    #####################################################
    # Adding the geometrical information of layer L1 to the 'layer_geometrics' database.
    L1_x = 0
    L1_y = L2_L3_y + L2_L3_height
    L1_z = 0
    layer_geometrics['L1_block'] = {"width": width_layers,
                                    "height": L1_height,
                                    "depth": depth_layers,
                                    "bottom_left_position": [L1_x, L1_y, L1_z],
                                    "num_excitatory_neurons": layer_data['L1']['num_excitatory_neurons'],
                                    "num_inhibitory_neurons": layer_data['L1']['num_inhibitory_neurons']
                                    }
    
    
    ############################################################
    ########  Generating Random Points for Every Layer  ########
    ############################################################
    # Initializing two dictionaries that will for each layer store the positions of the excitatory and inhibitory neurons.
    excitatory_positions = {}
    inhibitory_positions = {}
    
    ########
    ## L1 ##
    ########   
    # Generating a set of random x, y, and z positions that fall within layer L1 for the excitatory neurons.
    L1_x_exc = [random.uniform(layer_geometrics['L1_block']['bottom_left_position'][0], layer_geometrics['L1_block']['bottom_left_position'][0] + layer_geometrics['L1_block']['width']) for _ in range(layer_geometrics['L1_block']['num_excitatory_neurons'])]
    L1_y_exc = [random.uniform(layer_geometrics['L1_block']['bottom_left_position'][1], layer_geometrics['L1_block']['bottom_left_position'][1] + layer_geometrics['L1_block']['height']) for _ in range(layer_geometrics['L1_block']['num_excitatory_neurons'])]
    L1_z_exc = [random.uniform(layer_geometrics['L1_block']['bottom_left_position'][2], layer_geometrics['L1_block']['bottom_left_position'][2] + layer_geometrics['L1_block']['depth']) for _ in range(layer_geometrics['L1_block']['num_excitatory_neurons'])]
    L1_excitatory_topology = np.array([L1_x_exc, L1_y_exc, L1_z_exc])

    # Generating a set of random x, y, and z positions that fall within layer L1 for the inhibitory neurons.
    L1_x_inh = [random.uniform(layer_geometrics['L1_block']['bottom_left_position'][0], layer_geometrics['L1_block']['bottom_left_position'][0] + layer_geometrics['L1_block']['width']) for _ in range(layer_geometrics['L1_block']['num_inhibitory_neurons'])]
    L1_y_inh = [random.uniform(layer_geometrics['L1_block']['bottom_left_position'][1], layer_geometrics['L1_block']['bottom_left_position'][1] + layer_geometrics['L1_block']['height']) for _ in range(layer_geometrics['L1_block']['num_inhibitory_neurons'])]
    L1_z_inh = [random.uniform(layer_geometrics['L1_block']['bottom_left_position'][2], layer_geometrics['L1_block']['bottom_left_position'][2] + layer_geometrics['L1_block']['depth']) for _ in range(layer_geometrics['L1_block']['num_inhibitory_neurons'])]
    L1_inhibitory_topology = np.array([L1_x_inh, L1_y_inh, L1_z_inh])

    # Adding the excitatory and inhibitory topologies of the L1 layer to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['L1'] = L1_excitatory_topology
    inhibitory_positions['L1'] = L1_inhibitory_topology

    
    ########
    ## L2_L3 ##
    ########   
    # Generating a set of random x, y, and z positions that fall within layer L2_L3 for the excitatory neurons.
    L2_L3_x_exc = [random.uniform(layer_geometrics['L2_L3_block']['bottom_left_position'][0], layer_geometrics['L2_L3_block']['bottom_left_position'][0] + layer_geometrics['L2_L3_block']['width']) for _ in range(layer_geometrics['L2_L3_block']['num_excitatory_neurons'])]
    L2_L3_y_exc = [random.uniform(layer_geometrics['L2_L3_block']['bottom_left_position'][1], layer_geometrics['L2_L3_block']['bottom_left_position'][1] + layer_geometrics['L2_L3_block']['height']) for _ in range(layer_geometrics['L2_L3_block']['num_excitatory_neurons'])]
    L2_L3_z_exc = [random.uniform(layer_geometrics['L2_L3_block']['bottom_left_position'][2], layer_geometrics['L2_L3_block']['bottom_left_position'][2] + layer_geometrics['L2_L3_block']['depth']) for _ in range(layer_geometrics['L2_L3_block']['num_excitatory_neurons'])]
    L2_L3_excitatory_topology = np.array([L2_L3_x_exc, L2_L3_y_exc, L2_L3_z_exc])

    # Generating a set of random x, y, and z positions that fall within layer L2_L3 for the inhibitory neurons.
    L2_L3_x_inh = [random.uniform(layer_geometrics['L2_L3_block']['bottom_left_position'][0], layer_geometrics['L2_L3_block']['bottom_left_position'][0] + layer_geometrics['L2_L3_block']['width']) for _ in range(layer_geometrics['L2_L3_block']['num_inhibitory_neurons'])]
    L2_L3_y_inh = [random.uniform(layer_geometrics['L2_L3_block']['bottom_left_position'][1], layer_geometrics['L2_L3_block']['bottom_left_position'][1] + layer_geometrics['L2_L3_block']['height']) for _ in range(layer_geometrics['L2_L3_block']['num_inhibitory_neurons'])]
    L2_L3_z_inh = [random.uniform(layer_geometrics['L2_L3_block']['bottom_left_position'][2], layer_geometrics['L2_L3_block']['bottom_left_position'][2] + layer_geometrics['L2_L3_block']['depth']) for _ in range(layer_geometrics['L2_L3_block']['num_inhibitory_neurons'])]
    L2_L3_inhibitory_topology = np.array([L2_L3_x_inh, L2_L3_y_inh, L2_L3_z_inh])

    # Adding the excitatory and inhibitory topologies of the L2_L3 layer to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['L2_L3'] = L2_L3_excitatory_topology
    inhibitory_positions['L2_L3'] = L2_L3_inhibitory_topology

        
    ########
    ## L4 ##
    ########   
    # Generating a set of random x, y, and z positions that fall within layer L4 for the excitatory neurons.
    L4_x_exc = [random.uniform(layer_geometrics['L4_block']['bottom_left_position'][0], layer_geometrics['L4_block']['bottom_left_position'][0] + layer_geometrics['L4_block']['width']) for _ in range(layer_geometrics['L4_block']['num_excitatory_neurons'])]
    L4_y_exc = [random.uniform(layer_geometrics['L4_block']['bottom_left_position'][1], layer_geometrics['L4_block']['bottom_left_position'][1] + layer_geometrics['L4_block']['height']) for _ in range(layer_geometrics['L4_block']['num_excitatory_neurons'])]
    L4_z_exc = [random.uniform(layer_geometrics['L4_block']['bottom_left_position'][2], layer_geometrics['L4_block']['bottom_left_position'][2] + layer_geometrics['L4_block']['depth']) for _ in range(layer_geometrics['L4_block']['num_excitatory_neurons'])]
    L4_excitatory_topology = np.array([L4_x_exc, L4_y_exc, L4_z_exc])

    # Generating a set of random x, y, and z positions that fall within layer L4 for the inhibitory neurons.
    L4_x_inh = [random.uniform(layer_geometrics['L4_block']['bottom_left_position'][0], layer_geometrics['L4_block']['bottom_left_position'][0] + layer_geometrics['L4_block']['width']) for _ in range(layer_geometrics['L4_block']['num_inhibitory_neurons'])]
    L4_y_inh = [random.uniform(layer_geometrics['L4_block']['bottom_left_position'][1], layer_geometrics['L4_block']['bottom_left_position'][1] + layer_geometrics['L4_block']['height']) for _ in range(layer_geometrics['L4_block']['num_inhibitory_neurons'])]
    L4_z_inh = [random.uniform(layer_geometrics['L4_block']['bottom_left_position'][2], layer_geometrics['L4_block']['bottom_left_position'][2] + layer_geometrics['L4_block']['depth']) for _ in range(layer_geometrics['L4_block']['num_inhibitory_neurons'])]
    L4_inhibitory_topology = np.array([L4_x_inh, L4_y_inh, L4_z_inh])

    # Adding the excitatory and inhibitory topologies of the L4 layer to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['L4'] = L4_excitatory_topology
    inhibitory_positions['L4'] = L4_inhibitory_topology

    
    ########
    ## L5 ##
    ########   
    # Generating a set of random x, y, and z positions that fall within layer L5 for the excitatory neurons.
    L5_x_exc = [random.uniform(layer_geometrics['L5_block']['bottom_left_position'][0], layer_geometrics['L5_block']['bottom_left_position'][0] + layer_geometrics['L5_block']['width']) for _ in range(layer_geometrics['L5_block']['num_excitatory_neurons'])]
    L5_y_exc = [random.uniform(layer_geometrics['L5_block']['bottom_left_position'][1], layer_geometrics['L5_block']['bottom_left_position'][1] + layer_geometrics['L5_block']['height']) for _ in range(layer_geometrics['L5_block']['num_excitatory_neurons'])]
    L5_z_exc = [random.uniform(layer_geometrics['L5_block']['bottom_left_position'][2], layer_geometrics['L5_block']['bottom_left_position'][2] + layer_geometrics['L5_block']['depth']) for _ in range(layer_geometrics['L5_block']['num_excitatory_neurons'])]
    L5_excitatory_topology = np.array([L5_x_exc, L5_y_exc, L5_z_exc])

    # Generating a set of random x, y, and z positions that fall within layer L5 for the inhibitory neurons.
    L5_x_inh = [random.uniform(layer_geometrics['L5_block']['bottom_left_position'][0], layer_geometrics['L5_block']['bottom_left_position'][0] + layer_geometrics['L5_block']['width']) for _ in range(layer_geometrics['L5_block']['num_inhibitory_neurons'])]
    L5_y_inh = [random.uniform(layer_geometrics['L5_block']['bottom_left_position'][1], layer_geometrics['L5_block']['bottom_left_position'][1] + layer_geometrics['L5_block']['height']) for _ in range(layer_geometrics['L5_block']['num_inhibitory_neurons'])]
    L5_z_inh = [random.uniform(layer_geometrics['L5_block']['bottom_left_position'][2], layer_geometrics['L5_block']['bottom_left_position'][2] + layer_geometrics['L5_block']['depth']) for _ in range(layer_geometrics['L5_block']['num_inhibitory_neurons'])]
    L5_inhibitory_topology = np.array([L5_x_inh, L5_y_inh, L5_z_inh])

    # Adding the excitatory and inhibitory topologies of the L5 layer to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['L5'] = L5_excitatory_topology
    inhibitory_positions['L5'] = L5_inhibitory_topology

    
    ########
    ## L6 ##
    ########   
    # Generating a set of random x, y, and z positions that fall within layer L6 for the excitatory neurons.
    L6_x_exc = [random.uniform(layer_geometrics['L6_block']['bottom_left_position'][0], layer_geometrics['L6_block']['bottom_left_position'][0] + layer_geometrics['L6_block']['width']) for _ in range(layer_geometrics['L6_block']['num_excitatory_neurons'])]
    L6_y_exc = [random.uniform(layer_geometrics['L6_block']['bottom_left_position'][1], layer_geometrics['L6_block']['bottom_left_position'][1] + layer_geometrics['L6_block']['height']) for _ in range(layer_geometrics['L6_block']['num_excitatory_neurons'])]
    L6_z_exc = [random.uniform(layer_geometrics['L6_block']['bottom_left_position'][2], layer_geometrics['L6_block']['bottom_left_position'][2] + layer_geometrics['L6_block']['depth']) for _ in range(layer_geometrics['L6_block']['num_excitatory_neurons'])]
    L6_excitatory_topology = np.array([L6_x_exc, L6_y_exc, L6_z_exc])

    # Generating a set of random x, y, and z positions that fall within layer L6 for the inhibitory neurons.
    L6_x_inh = [random.uniform(layer_geometrics['L6_block']['bottom_left_position'][0], layer_geometrics['L6_block']['bottom_left_position'][0] + layer_geometrics['L6_block']['width']) for _ in range(layer_geometrics['L6_block']['num_inhibitory_neurons'])]
    L6_y_inh = [random.uniform(layer_geometrics['L6_block']['bottom_left_position'][1], layer_geometrics['L6_block']['bottom_left_position'][1] + layer_geometrics['L6_block']['height']) for _ in range(layer_geometrics['L6_block']['num_inhibitory_neurons'])]
    L6_z_inh = [random.uniform(layer_geometrics['L6_block']['bottom_left_position'][2], layer_geometrics['L6_block']['bottom_left_position'][2] + layer_geometrics['L6_block']['depth']) for _ in range(layer_geometrics['L6_block']['num_inhibitory_neurons'])]
    L6_inhibitory_topology = np.array([L6_x_inh, L6_y_inh, L6_z_inh])

    # Adding the excitatory and inhibitory topologies of the L6 layer to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['L6'] = L6_excitatory_topology
    inhibitory_positions['L6'] = L6_inhibitory_topology

    print(layer_geometrics)

    
    return layer_data, layer_geometrics, excitatory_positions, inhibitory_positions

<br></br>

In [10]:
# Creates the group of excitatory neurons.
def create_group_py(topology, noise, masks, current_layer, stimulus_layers, treatment_layers, excitatory_topologies_lengths, group_name='exc_group', integ_method='exponential_euler'):

    # Extracting the passed on parameters.
    x, y, z = topology
    mu_noise, sigma_noise = noise
    stimulus_mask, treatment_mask = masks

    # Adjusting the group name.
    group_name = group_name + "_" + current_layer

    # Initializing a group of excitatory neurons with the following parameters:
    # - py_eqs => Differential equations that define the behavior of the excitatory neuron.
    # - threshold => Neurons fire an action potential when their membrane potential 'v' exceeds the threshold 'V_th'.
    # - reset => Resets neuron states after they fire according to 'reset_eqs'.
    # - refractory => Sets a refractory period during which an excitatory neuron cannot fire again.
    # - method => Specifies the integration method for solving the differential equations.
    G_exc = NeuronGroup(len(x),py_eqs,threshold='v>V_th',reset=reset_eqs,refractory=3*ms,name=group_name, method=integ_method)

    # Initializing the membrane potential 'v' to be a random value between -60 mV and -100 mV.
    G_exc.v = '-60*mvolt-rand()*40*mvolt'

    # Sets the neurotransmitter to be used to be glutamate (which is an excitatory neurotransmitter).
    G_exc.glu = 1

    # Assigns the positions of the neurons in micrometers.
    G_exc.x = x * um
    G_exc.y = y * um
    G_exc.z = z * um

    # Sets the size of the neurons to 'taille_exc_normale'.
    G_exc.taille = taille_exc_normale

    # Sets the mean and standard deviation of the noise affecting the neurons.
    G_exc.mu_noise = mu_noise
    G_exc.sigma_noise = sigma_noise

    ### FOR THE MASKS: Optionally allow for multiple layers to be part of the treatment/stimulus mask.
    # Applying the treatment mask to the neuron by first checking whether the current layer is in the list of treatment layers. If not, a list of zeros is added as treatment mask.
    if current_layer in treatment_layers:
        G_exc.treatment_mask = treatment_mask
    else:
        G_exc.treatment_mask = np.zeros(len(x))

    # Applying the stimulus mask to the neuron by first checking whether the current layer is in the list of stimulus layers. If not, a list of zeros is added as stimulus mask.
    if current_layer in stimulus_layers:
        G_exc.stimulus_mask = stimulus_mask
    else:
        G_exc.stimulus_mask = np.zeros(len(x))
        
    return G_exc

<br></br>

In [11]:
# Creates the group of inhibitory neurons.
def create_group_inh(topology, noise, treatment_mask, current_layer, treatment_layers, inhibitory_topologies_lengths, group_name='inh_group', integ_method='exponential_euler'):

    # Extracting the passed on parameters.
    x, y, z = topology
    mu_noise, sigma_noise = noise

    # Adjusting the group name.
    group_name = group_name + "_" + current_layer

    # Initializing a group of inhibitory neurons with the following parameters:
    # - inh_eqs => Differential equations that define the behavior of the inhibitory neuron.
    # - threshold => Neurons fire an action potential when their membrane potential 'v' exceeds the threshold 'V_th'.
    # - refractory => Sets a refractory period during which an inhibitory neuron cannot fire again.
    # - method => Specifies the integration method for solving the differential equations.
    G_inh = NeuronGroup(len(x),inh_eqs,threshold='v>V_th', name=group_name, refractory=3*ms,method=integ_method)

    # Initializing the membrane potential 'v' to be a random value between -60 mV and -70 mV.
    G_inh.v = -60*mvolt-rand()*10*mvolt

    # Sets the size of the neurons to 'taille_exc_normale'.
    G_inh.taille = taille_inh_normale

    # Assigns the positions of the neurons in micrometers.
    G_inh.x = x * um
    G_inh.y = y * um
    G_inh.z = z * um

    # Sets the mean and standard deviation of the noise affecting the neurons.
    G_inh.mu_noise = mu_noise
    G_inh.sigma_noise = sigma_noise

    ### FOR THE MASKS: Optionally allow for multiple layers to be part of the treatment mask.
    # Applying the treatment mask to the neuron by first checking whether the current layer is in the list of treatment layers. If not, a list of zeros is added as treatment mask.
    if current_layer in treatment_layers:
        G_inh.treatment_mask = treatment_mask
    else:
        G_inh.treatment_mask = np.zeros(len(x))

    return G_inh

<br></br>

In [12]:
# Creates a group for Local Field Potential (LFP) recording.
def create_group_lfp(LFP_electrode_layer, layer_geometrics):
    
    # Setting up a singular LFP electrode
    Ne = 1

    # Setting the resistivity of the extracellular field to 0.3 siemens per meter, which is within the typical range for biological tissue (0.3-0.4 S/m).
    sigma = 0.3*siemens/meter

    # Initializes a group of neurons, which will only consist of 1 representing a single LFP electrode which has three state variables:
    # - v : volt => Represents the voltage (LFP signal).
    # - x : meter => Represent the x-coordinate of the electrode.
    # - y : meter => Represent the y-coordinate of the electrode.
    # - z : meter => Represent the z-coordinate of the electrode.
    lfp = NeuronGroup(Ne, model='''v : volt
                                   x : meter
                                   y : meter
                                   z : meter''')

    # Defining the x, y, and z coordinates of the LFP electrode based on what the layer the LFP electrode is in.
    if LFP_electrode_layer == "L1":
        lfp.x = (layer_geometrics['L1_block']['bottom_left_position'][0] + (layer_geometrics['L1_block']['width'] / 2)) * mm
        lfp.y = (layer_geometrics['L1_block']['bottom_left_position'][1] + (layer_geometrics['L1_block']['height'] / 2)) * mm
        lfp.z = (layer_geometrics['L1_block']['bottom_left_position'][2] + (layer_geometrics['L1_block']['depth'] / 2)) * mm
    if LFP_electrode_layer == "L2_L3":
        lfp.x = (layer_geometrics['L2_L3_block']['bottom_left_position'][0] + (layer_geometrics['L2_L3_block']['width'] / 2)) * mm
        lfp.y = (layer_geometrics['L2_L3_block']['bottom_left_position'][1] + (layer_geometrics['L2_L3_block']['height'] / 2)) * mm
        lfp.z = (layer_geometrics['L2_L3_block']['bottom_left_position'][2] + (layer_geometrics['L2_L3_block']['depth'] / 2)) * mm
    if LFP_electrode_layer == "L4":
        lfp.x = (layer_geometrics['L4_block']['bottom_left_position'][0] + (layer_geometrics['L4_block']['width'] / 2)) * mm
        lfp.y = (layer_geometrics['L4_block']['bottom_left_position'][1] + (layer_geometrics['L4_block']['height'] / 2)) * mm
        lfp.z = (layer_geometrics['L4_block']['bottom_left_position'][2] + (layer_geometrics['L4_block']['depth'] / 2)) * mm
    if LFP_electrode_layer == "L5":
        lfp.x = (layer_geometrics['L5_block']['bottom_left_position'][0] + (layer_geometrics['L5_block']['width'] / 2)) * mm
        lfp.y = (layer_geometrics['L5_block']['bottom_left_position'][1] + (layer_geometrics['L5_block']['height'] / 2)) * mm
        lfp.z = (layer_geometrics['L5_block']['bottom_left_position'][2] + (layer_geometrics['L5_block']['depth'] / 2)) * mm
    if LFP_electrode_layer == "L6":
        lfp.x = (layer_geometrics['L6_block']['bottom_left_position'][0] + (layer_geometrics['L6_block']['width'] / 2)) * mm
        lfp.y = (layer_geometrics['L6_block']['bottom_left_position'][1] + (layer_geometrics['L6_block']['height'] / 2)) * mm
        lfp.z = (layer_geometrics['L6_block']['bottom_left_position'][2] + (layer_geometrics['L6_block']['depth'] / 2)) * mm

    return lfp

<br></br><br></br>

<h2 style="font-size: 40px;">Network Configuration Function</h2>

In [13]:
# Prepares the network of neurons. The topology is already there, but the neurons still need to be connected, which is done here.
def prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables):
    
    # Extracting the connection probabilities between neurons from the same layer and from different layers.
    connection_probabilities = current_variables['p']

    # Extracting the topologies from the passed on parameter 'topologies'.
    topology_L1_exc, topology_L2_L3_exc, topology_L4_exc, topology_L5_exc, topology_L6_exc, topology_L1_inh, topology_L2_L3_inh, topology_L4_inh, topology_L5_inh, topology_L6_inh = topologies

    # Extracting the treatment masks from the passed on parameter 'treatment_masks'.
    treatment_mask_exc, treatment_mask_inh = treatment_masks

    # Calculating the lengths of the excitatory and inhibitory topologies.
    excitatory_topologies_lengths = [len(topology_L1_exc[0]), len(topology_L2_L3_exc[0]), len(topology_L4_exc[0]), len(topology_L5_exc[0]), len(topology_L6_exc[0])]
    inhibitory_topologies_lengths = [len(topology_L1_inh[0]), len(topology_L2_L3_inh[0]), len(topology_L4_inh[0]), len(topology_L5_inh[0]), len(topology_L6_inh[0])]

    
    ##########################################
    ########  Creating Neuron Groups  ########
    ##########################################
    # Creating excitatory neuron groups for each layer.
    G_L1_exc = create_group_py(topology_L1_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'L1', current_variables['stimulus_layers'], current_variables['treatment_layers'], excitatory_topologies_lengths)
    G_L2_L3_exc = create_group_py(topology_L2_L3_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'L2_L3', current_variables['stimulus_layers'], current_variables['treatment_layers'], excitatory_topologies_lengths)
    G_L4_exc = create_group_py(topology_L4_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'L4', current_variables['stimulus_layers'], current_variables['treatment_layers'], excitatory_topologies_lengths)
    G_L5_exc = create_group_py(topology_L5_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'L5', current_variables['stimulus_layers'], current_variables['treatment_layers'], excitatory_topologies_lengths)
    G_L6_exc = create_group_py(topology_L6_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'L6', current_variables['stimulus_layers'], current_variables['treatment_layers'], excitatory_topologies_lengths)

    # Creating inhibitory neuron groups for each layer.
    G_L1_inh = create_group_inh(topology_L1_inh, current_variables['noise_inh'], treatment_mask_inh, 'L1', current_variables['treatment_layers'], inhibitory_topologies_lengths)
    G_L2_L3_inh = create_group_inh(topology_L2_L3_inh, current_variables['noise_inh'], treatment_mask_inh, 'L2_L3', current_variables['treatment_layers'], inhibitory_topologies_lengths)
    G_L4_inh = create_group_inh(topology_L4_inh, current_variables['noise_inh'], treatment_mask_inh, 'L4', current_variables['treatment_layers'], inhibitory_topologies_lengths)
    G_L5_inh = create_group_inh(topology_L5_inh, current_variables['noise_inh'], treatment_mask_inh, 'L5', current_variables['treatment_layers'], inhibitory_topologies_lengths)
    G_L6_inh = create_group_inh(topology_L6_inh, current_variables['noise_inh'], treatment_mask_inh, 'L6', current_variables['treatment_layers'], inhibitory_topologies_lengths)

    # Setting up a Local Field Potential (LFP) electrode.
    G_lfp = create_group_lfp(current_variables['LFP_electrode_layer'], current_variables['layer_geometrics'])
    
    # Adding the neuron groups to a list.
    neuron_groups = [G_L1_exc, G_L2_L3_exc, G_L4_exc, G_L5_exc, G_L6_exc, G_L1_inh, G_L2_L3_inh, G_L4_inh, G_L5_inh, G_L6_inh, G_lfp]

    
    #####################################
    ########  Creating Monitors  ########
    #####################################
    # Setting up monitors that monitor the population firing rate of all excitatory and inhibitory neuron groups.
    popmon_L1_exc = PopulationRateMonitor(G_L1_exc)
    popmon_L2_L3_exc = PopulationRateMonitor(G_L2_L3_exc)
    popmon_L4_exc = PopulationRateMonitor(G_L4_exc)
    popmon_L5_exc = PopulationRateMonitor(G_L5_exc)
    popmon_L6_exc = PopulationRateMonitor(G_L6_exc)
    popmon_L1_inh = PopulationRateMonitor(G_L1_inh)
    popmon_L2_L3_inh = PopulationRateMonitor(G_L2_L3_inh)
    popmon_L4_inh = PopulationRateMonitor(G_L4_inh)
    popmon_L5_inh = PopulationRateMonitor(G_L5_inh)
    popmon_L6_inh = PopulationRateMonitor(G_L6_inh)

    # Setting up a monitor that monitors the voltage of the LFP group.
    Mlfp = StateMonitor(G_lfp, 'v', record=True)

    # Setting up monitors that monitor the membrane potential, the noise current (and the stimulus current) for the selected excitatory/inhibitory neurons.
    statemon_L1_exc = StateMonitor(G_L1_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L2_L3_exc = StateMonitor(G_L2_L3_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L4_exc = StateMonitor(G_L4_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L5_exc = StateMonitor(G_L5_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L6_exc = StateMonitor(G_L6_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L1_inh = StateMonitor(G_L1_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L2_L3_inh = StateMonitor(G_L2_L3_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L4_inh = StateMonitor(G_L4_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L5_inh = StateMonitor(G_L5_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_L6_inh = StateMonitor(G_L6_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    
    monitors = [popmon_L1_exc, popmon_L2_L3_exc, popmon_L4_exc, popmon_L5_exc, popmon_L6_exc, popmon_L1_inh, popmon_L2_L3_inh, popmon_L4_inh, popmon_L5_inh, popmon_L6_inh, Mlfp, statemon_L1_exc, statemon_L2_L3_exc, statemon_L4_exc, statemon_L5_exc, statemon_L6_exc, statemon_L1_inh, statemon_L2_L3_inh, statemon_L4_inh, statemon_L5_inh, statemon_L6_inh]


    ############################################
    ########  Connecting Neuron Groups  ########
    ############################################
    # Configuring synaptic connections between neuron groups where in the function call 'Synapses()' the first parameter is the pre-synaptic group and the second parameter is the post-synaptic group. 
    # The third parameter 'on_pre' specifies the action to be taken when a pre-synaptic neuron fires where the synaptic event will increase the post-synaptic event by a certain amount defined by:
    # - gain => A scaling factor for the synaptic strength.
    # - g_max/siemens => The maximum conductance for the type of synapse normalized by dividing by 'siemens' which is a unit of conductance.
    # - glu_pre => The amount of glutamate released from the pre-synaptic neuron upon firing.
    # MIND: This does not create synapses but instead only specifies their dynamics. The actual synapse connections are created by using the function 'connect()'.
    synapses = []
    
    ################################      
    ## Connections Within layers ##
    ################################
    # Generating within the L1 layer the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_L1_e2e = Synapses(G_L1_exc, G_L1_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L1_exc_e2e')
    S_L1_e2i = Synapses(G_L1_exc, G_L1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L1_exc_e2i')
    S_L1_i2e = Synapses(G_L1_inh, G_L1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L1_exc_i2e')
    S_L1_i2i = Synapses(G_L1_inh, G_L1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L1_exc_i2i')
    S_L1_e2e.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L1_e2i.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L1_i2e.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L1_i2i.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_L1_e2e)
    synapses.append(S_L1_e2i)
    synapses.append(S_L1_i2e)
    synapses.append(S_L1_i2i)

    # Generating within the L2_L3 layer the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_L2_L3_e2e = Synapses(G_L2_L3_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L2_L3_exc_e2e')
    S_L2_L3_e2i = Synapses(G_L2_L3_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L2_L3_exc_e2i')
    S_L2_L3_i2e = Synapses(G_L2_L3_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L2_L3_exc_i2e')
    S_L2_L3_i2i = Synapses(G_L2_L3_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L2_L3_exc_i2i')
    S_L2_L3_e2e.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L2_L3_e2i.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L2_L3_i2e.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L2_L3_i2i.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_L2_L3_e2e)
    synapses.append(S_L2_L3_e2i)
    synapses.append(S_L2_L3_i2e)
    synapses.append(S_L2_L3_i2i)

    # Generating within the L4 layer the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_L4_e2e = Synapses(G_L4_exc, G_L4_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L4_exc_e2e')
    S_L4_e2i = Synapses(G_L4_exc, G_L4_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L4_exc_e2i')
    S_L4_i2e = Synapses(G_L4_inh, G_L4_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L4_exc_i2e')
    S_L4_i2i = Synapses(G_L4_inh, G_L4_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L4_exc_i2i')
    S_L4_e2e.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L4_e2i.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L4_i2e.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L4_i2i.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_L4_e2e)
    synapses.append(S_L4_e2i)
    synapses.append(S_L4_i2e)
    synapses.append(S_L4_i2i)

    # Generating within the L5 layer the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_L5_e2e = Synapses(G_L5_exc, G_L5_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L5_exc_e2e')
    S_L5_e2i = Synapses(G_L5_exc, G_L5_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L5_exc_e2i')
    S_L5_i2e = Synapses(G_L5_inh, G_L5_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L5_exc_i2e')
    S_L5_i2i = Synapses(G_L5_inh, G_L5_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L5_exc_i2i')
    S_L5_e2e.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L5_e2i.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L5_i2e.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L5_i2i.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_L5_e2e)
    synapses.append(S_L5_e2i)
    synapses.append(S_L5_i2e)
    synapses.append(S_L5_i2i)

    # Generating within the L6 layer the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_L6_e2e = Synapses(G_L6_exc, G_L6_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L6_exc_e2e')
    S_L6_e2i = Synapses(G_L6_exc, G_L6_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_G_L6_exc_e2i')
    S_L6_i2e = Synapses(G_L6_inh, G_L6_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L6_exc_i2e')
    S_L6_i2i = Synapses(G_L6_inh, G_L6_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_G_L6_exc_i2i')
    S_L6_e2e.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L6_e2i.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L6_i2e.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L6_i2i.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_L6_e2e)
    synapses.append(S_L6_e2i)
    synapses.append(S_L6_i2e)
    synapses.append(S_L6_i2i)



    #################################      
    ## Connections Between layers ##
    #################################
    # Generating from the L1 to the L2_L3 layer the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_L1_L2_L3_e2e = Synapses(G_L1_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_G_L1_and_L2_L3_exc_e2e')
    S_L1_L2_L3_e2i = Synapses(G_L1_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_G_L1_and_L2_L3_exc_e2i')
    S_L1_L2_L3_i2e = Synapses(G_L1_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_G_L1_and_L2_L3_exc_i2e')
    S_L1_L2_L3_i2i = Synapses(G_L1_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_G_L1_and_L2_L3_exc_i2i')
    S_L1_L2_L3_e2e.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L1_L2_L3_e2i.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L1_L2_L3_i2e.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L1_L2_L3_i2i.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_L1_L2_L3_e2e)
    synapses.append(S_L1_L2_L3_e2i)
    synapses.append(S_L1_L2_L3_i2e)
    synapses.append(S_L1_L2_L3_i2i)

    # Generating from the L4 to the L2_L3 layer the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_L4_L2_L3_e2e = Synapses(G_L4_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_G_L4_and_L2_L3_exc_e2e')
    S_L4_L2_L3_e2i = Synapses(G_L4_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_G_L4_and_L2_L3_exc_e2i')
    S_L4_L2_L3_i2e = Synapses(G_L4_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_G_L4_and_L2_L3_exc_i2e')
    S_L4_L2_L3_i2i = Synapses(G_L4_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_G_L4_and_L2_L3_exc_i2i')
    S_L4_L2_L3_e2e.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L4_L2_L3_e2i.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L4_L2_L3_i2e.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L4_L2_L3_i2i.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_L4_L2_L3_e2e)
    synapses.append(S_L4_L2_L3_e2i)
    synapses.append(S_L4_L2_L3_i2e)
    synapses.append(S_L4_L2_L3_i2i)

    # Generating from the L5 to the L6 layer the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_L5_L6_e2e = Synapses(G_L5_exc, G_L6_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_G_L5_and_L6_exc_e2e')
    S_L5_L6_e2i = Synapses(G_L5_exc, G_L6_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_G_L5_and_L6_exc_e2i')
    S_L5_L6_i2e = Synapses(G_L5_inh, G_L6_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_G_L5_and_L6_exc_i2e')
    S_L5_L6_i2i = Synapses(G_L5_inh, G_L6_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_G_L5_and_L6_exc_i2i')
    S_L5_L6_e2e.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L5_L6_e2i.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L5_L6_i2e.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_L5_L6_i2i.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_L5_L6_e2e)
    synapses.append(S_L5_L6_e2i)
    synapses.append(S_L5_L6_i2e)
    synapses.append(S_L5_L6_i2i)

    
    # Since we want the total number of connections from the L1 to the L5 layer to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_L1_L5_e2e_no_dist = Synapses(G_L1_exc, G_L5_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L1_L5_e2e_dist = Synapses(G_L1_exc, G_L5_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L1_L5_e2e_no_dist.connect(p=current_variables['p'][0])
    testing_S_L1_L5_e2e_dist.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L1_L5_e2e = len(testing_S_L1_L5_e2e_dist.N_outgoing_pre) / len(testing_S_L1_L5_e2e_no_dist.N_outgoing_pre)
    
    testing_S_L1_L5_e2i_no_dist = Synapses(G_L1_exc, G_L5_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L1_L5_e2i_dist = Synapses(G_L1_exc, G_L5_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L1_L5_e2i_no_dist.connect(p=current_variables['p'][1])
    testing_S_L1_L5_e2i_dist.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L1_L5_e2i = len(testing_S_L1_L5_e2i_dist.N_outgoing_pre) / len(testing_S_L1_L5_e2i_no_dist.N_outgoing_pre)

    testing_S_L1_L5_i2e_no_dist = Synapses(G_L1_inh, G_L5_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L1_L5_i2e_dist = Synapses(G_L1_inh, G_L5_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L1_L5_i2e_no_dist.connect(p=current_variables['p'][2])
    testing_S_L1_L5_i2e_dist.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L1_L5_i2e = len(testing_S_L1_L5_i2e_dist.N_outgoing_pre) / len(testing_S_L1_L5_i2e_no_dist.N_outgoing_pre)

    testing_S_L1_L5_i2i_no_dist = Synapses(G_L1_inh, G_L5_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L1_L5_i2i_dist = Synapses(G_L1_inh, G_L5_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L1_L5_i2i_no_dist.connect(p=current_variables['p'][3])
    testing_S_L1_L5_i2i_dist.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L1_L5_i2i = len(testing_S_L1_L5_i2i_dist.N_outgoing_pre) / len(testing_S_L1_L5_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_L1_L5_e2e = Synapses(G_L1_exc, G_L5_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L1_L5_e2e')
    S_L1_L5_e2i = Synapses(G_L1_exc, G_L5_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L1_L5_e2i')
    S_L1_L5_i2e = Synapses(G_L1_inh, G_L5_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L1_L5_i2e')
    S_L1_L5_i2i = Synapses(G_L1_inh, G_L5_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L1_L5_i2i')
    S_L1_L5_e2e.connect(p=current_variables['p'][0]*ratio_L1_L5_e2e)
    S_L1_L5_e2i.connect(p=current_variables['p'][1]*ratio_L1_L5_e2i)
    S_L1_L5_i2e.connect(p=current_variables['p'][2]*ratio_L1_L5_i2e)
    S_L1_L5_i2i.connect(p=current_variables['p'][3]*ratio_L1_L5_i2i)
    synapses.append(S_L1_L5_e2e)
    synapses.append(S_L1_L5_e2i)
    synapses.append(S_L1_L5_i2e)
    synapses.append(S_L1_L5_i2i)


    # Since we want the total number of connections from the L2_L3 to the L5 layer to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_L2_L3_L5_e2e_no_dist = Synapses(G_L2_L3_exc, G_L5_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L2_L3_L5_e2e_dist = Synapses(G_L2_L3_exc, G_L5_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L2_L3_L5_e2e_no_dist.connect(p=current_variables['p'][0])
    testing_S_L2_L3_L5_e2e_dist.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L2_L3_L5_e2e = len(testing_S_L2_L3_L5_e2e_dist.N_outgoing_pre) / len(testing_S_L2_L3_L5_e2e_no_dist.N_outgoing_pre)
    
    testing_S_L2_L3_L5_e2i_no_dist = Synapses(G_L2_L3_exc, G_L5_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L2_L3_L5_e2i_dist = Synapses(G_L2_L3_exc, G_L5_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L2_L3_L5_e2i_no_dist.connect(p=current_variables['p'][1])
    testing_S_L2_L3_L5_e2i_dist.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L2_L3_L5_e2i = len(testing_S_L2_L3_L5_e2i_dist.N_outgoing_pre) / len(testing_S_L2_L3_L5_e2i_no_dist.N_outgoing_pre)

    testing_S_L2_L3_L5_i2e_no_dist = Synapses(G_L2_L3_inh, G_L5_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L2_L3_L5_i2e_dist = Synapses(G_L2_L3_inh, G_L5_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L2_L3_L5_i2e_no_dist.connect(p=current_variables['p'][2])
    testing_S_L2_L3_L5_i2e_dist.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L2_L3_L5_i2e = len(testing_S_L2_L3_L5_i2e_dist.N_outgoing_pre) / len(testing_S_L2_L3_L5_i2e_no_dist.N_outgoing_pre)

    testing_S_L2_L3_L5_i2i_no_dist = Synapses(G_L2_L3_inh, G_L5_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L2_L3_L5_i2i_dist = Synapses(G_L2_L3_inh, G_L5_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L2_L3_L5_i2i_no_dist.connect(p=current_variables['p'][3])
    testing_S_L2_L3_L5_i2i_dist.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L2_L3_L5_i2i = len(testing_S_L2_L3_L5_i2i_dist.N_outgoing_pre) / len(testing_S_L2_L3_L5_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_L2_L3_L5_e2e = Synapses(G_L2_L3_exc, G_L5_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L2_L3_L5_e2e')
    S_L2_L3_L5_e2i = Synapses(G_L2_L3_exc, G_L5_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L2_L3_L5_e2i')
    S_L2_L3_L5_i2e = Synapses(G_L2_L3_inh, G_L5_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L2_L3_L5_i2e')
    S_L2_L3_L5_i2i = Synapses(G_L2_L3_inh, G_L5_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L2_L3_L5_i2i')
    S_L2_L3_L5_e2e.connect(p=current_variables['p'][0]*ratio_L2_L3_L5_e2e)
    S_L2_L3_L5_e2i.connect(p=current_variables['p'][1]*ratio_L2_L3_L5_e2i)
    S_L2_L3_L5_i2e.connect(p=current_variables['p'][2]*ratio_L2_L3_L5_i2e)
    S_L2_L3_L5_i2i.connect(p=current_variables['p'][3]*ratio_L2_L3_L5_i2i)
    synapses.append(S_L2_L3_L5_e2e)
    synapses.append(S_L2_L3_L5_e2i)
    synapses.append(S_L2_L3_L5_i2e)
    synapses.append(S_L2_L3_L5_i2i)


    # Since we want the total number of connections from the L4 to the L2_L3 layer to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_L4_L2_L3_e2e_no_dist = Synapses(G_L4_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L4_L2_L3_e2e_dist = Synapses(G_L4_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L4_L2_L3_e2e_no_dist.connect(p=current_variables['p'][0])
    testing_S_L4_L2_L3_e2e_dist.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L4_L2_L3_e2e = len(testing_S_L4_L2_L3_e2e_dist.N_outgoing_pre) / len(testing_S_L4_L2_L3_e2e_no_dist.N_outgoing_pre)
    
    testing_S_L4_L2_L3_e2i_no_dist = Synapses(G_L4_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L4_L2_L3_e2i_dist = Synapses(G_L4_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L4_L2_L3_e2i_no_dist.connect(p=current_variables['p'][1])
    testing_S_L4_L2_L3_e2i_dist.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L4_L2_L3_e2i = len(testing_S_L4_L2_L3_e2i_dist.N_outgoing_pre) / len(testing_S_L4_L2_L3_e2i_no_dist.N_outgoing_pre)

    testing_S_L4_L2_L3_i2e_no_dist = Synapses(G_L4_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L4_L2_L3_i2e_dist = Synapses(G_L4_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L4_L2_L3_i2e_no_dist.connect(p=current_variables['p'][2])
    testing_S_L4_L2_L3_i2e_dist.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L4_L2_L3_i2e = len(testing_S_L4_L2_L3_i2e_dist.N_outgoing_pre) / len(testing_S_L4_L2_L3_i2e_no_dist.N_outgoing_pre)

    testing_S_L4_L2_L3_i2i_no_dist = Synapses(G_L4_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L4_L2_L3_i2i_dist = Synapses(G_L4_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L4_L2_L3_i2i_no_dist.connect(p=current_variables['p'][3])
    testing_S_L4_L2_L3_i2i_dist.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L4_L2_L3_i2i = len(testing_S_L4_L2_L3_i2i_dist.N_outgoing_pre) / len(testing_S_L4_L2_L3_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_L4_L2_L3_e2e = Synapses(G_L4_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L4_L2_L3_e2e')
    S_L4_L2_L3_e2i = Synapses(G_L4_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L4_L2_L3_e2i')
    S_L4_L2_L3_i2e = Synapses(G_L4_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L4_L2_L3_i2e')
    S_L4_L2_L3_i2i = Synapses(G_L4_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L4_L2_L3_i2i')
    S_L4_L2_L3_e2e.connect(p=current_variables['p'][0]*ratio_L4_L2_L3_e2e)
    S_L4_L2_L3_e2i.connect(p=current_variables['p'][1]*ratio_L4_L2_L3_e2i)
    S_L4_L2_L3_i2e.connect(p=current_variables['p'][2]*ratio_L4_L2_L3_i2e)
    S_L4_L2_L3_i2i.connect(p=current_variables['p'][3]*ratio_L4_L2_L3_i2i)
    synapses.append(S_L4_L2_L3_e2e)
    synapses.append(S_L4_L2_L3_e2i)
    synapses.append(S_L4_L2_L3_i2e)
    synapses.append(S_L4_L2_L3_i2i)

    
    # Since we want the total number of connections from the L5 to the L1 layer to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_L5_L1_e2e_no_dist = Synapses(G_L5_exc, G_L1_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L5_L1_e2e_dist = Synapses(G_L5_exc, G_L1_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L5_L1_e2e_no_dist.connect(p=current_variables['p'][0])
    testing_S_L5_L1_e2e_dist.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L5_L1_e2e = len(testing_S_L5_L1_e2e_dist.N_outgoing_pre) / len(testing_S_L5_L1_e2e_no_dist.N_outgoing_pre)
    
    testing_S_L5_L1_e2i_no_dist = Synapses(G_L5_exc, G_L1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L5_L1_e2i_dist = Synapses(G_L5_exc, G_L1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L5_L1_e2i_no_dist.connect(p=current_variables['p'][1])
    testing_S_L5_L1_e2i_dist.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L5_L1_e2i = len(testing_S_L5_L1_e2i_dist.N_outgoing_pre) / len(testing_S_L5_L1_e2i_no_dist.N_outgoing_pre)

    testing_S_L5_L1_i2e_no_dist = Synapses(G_L5_inh, G_L1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L5_L1_i2e_dist = Synapses(G_L5_inh, G_L1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L5_L1_i2e_no_dist.connect(p=current_variables['p'][2])
    testing_S_L5_L1_i2e_dist.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L5_L1_i2e = len(testing_S_L5_L1_i2e_dist.N_outgoing_pre) / len(testing_S_L5_L1_i2e_no_dist.N_outgoing_pre)

    testing_S_L5_L1_i2i_no_dist = Synapses(G_L5_inh, G_L1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L5_L1_i2i_dist = Synapses(G_L5_inh, G_L1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L5_L1_i2i_no_dist.connect(p=current_variables['p'][3])
    testing_S_L5_L1_i2i_dist.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L5_L1_i2i = len(testing_S_L5_L1_i2i_dist.N_outgoing_pre) / len(testing_S_L5_L1_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_L5_L1_e2e = Synapses(G_L5_exc, G_L1_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L5_L1_e2e')
    S_L5_L1_e2i = Synapses(G_L5_exc, G_L1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L5_L1_e2i')
    S_L5_L1_i2e = Synapses(G_L5_inh, G_L1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L5_L1_i2e')
    S_L5_L1_i2i = Synapses(G_L5_inh, G_L1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L5_L1_i2i')
    S_L5_L1_e2e.connect(p=current_variables['p'][0]*ratio_L5_L1_e2e)
    S_L5_L1_e2i.connect(p=current_variables['p'][1]*ratio_L5_L1_e2i)
    S_L5_L1_i2e.connect(p=current_variables['p'][2]*ratio_L5_L1_i2e)
    S_L5_L1_i2i.connect(p=current_variables['p'][3]*ratio_L5_L1_i2i)
    synapses.append(S_L5_L1_e2e)
    synapses.append(S_L5_L1_e2i)
    synapses.append(S_L5_L1_i2e)
    synapses.append(S_L5_L1_i2i)


    # Since we want the total number of connections from the L5 to the L2_L3 layer to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_L5_L2_L3_e2e_no_dist = Synapses(G_L5_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L5_L2_L3_e2e_dist = Synapses(G_L5_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L5_L2_L3_e2e_no_dist.connect(p=current_variables['p'][0])
    testing_S_L5_L2_L3_e2e_dist.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L5_L2_L3_e2e = len(testing_S_L5_L2_L3_e2e_dist.N_outgoing_pre) / len(testing_S_L5_L2_L3_e2e_no_dist.N_outgoing_pre)
    
    testing_S_L5_L2_L3_e2i_no_dist = Synapses(G_L5_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L5_L2_L3_e2i_dist = Synapses(G_L5_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L5_L2_L3_e2i_no_dist.connect(p=current_variables['p'][1])
    testing_S_L5_L2_L3_e2i_dist.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L5_L2_L3_e2i = len(testing_S_L5_L2_L3_e2i_dist.N_outgoing_pre) / len(testing_S_L5_L2_L3_e2i_no_dist.N_outgoing_pre)

    testing_S_L5_L2_L3_i2e_no_dist = Synapses(G_L5_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L5_L2_L3_i2e_dist = Synapses(G_L5_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L5_L2_L3_i2e_no_dist.connect(p=current_variables['p'][2])
    testing_S_L5_L2_L3_i2e_dist.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L5_L2_L3_i2e = len(testing_S_L5_L2_L3_i2e_dist.N_outgoing_pre) / len(testing_S_L5_L2_L3_i2e_no_dist.N_outgoing_pre)

    testing_S_L5_L2_L3_i2i_no_dist = Synapses(G_L5_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L5_L2_L3_i2i_dist = Synapses(G_L5_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L5_L2_L3_i2i_no_dist.connect(p=current_variables['p'][3])
    testing_S_L5_L2_L3_i2i_dist.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L5_L2_L3_i2i = len(testing_S_L5_L2_L3_i2i_dist.N_outgoing_pre) / len(testing_S_L5_L2_L3_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_L5_L2_L3_e2e = Synapses(G_L5_exc, G_L2_L3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L5_L2_L3_e2e')
    S_L5_L2_L3_e2i = Synapses(G_L5_exc, G_L2_L3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L5_L2_L3_e2i')
    S_L5_L2_L3_i2e = Synapses(G_L5_inh, G_L2_L3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L5_L2_L3_i2e')
    S_L5_L2_L3_i2i = Synapses(G_L5_inh, G_L2_L3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L5_L2_L3_i2i')
    S_L5_L2_L3_e2e.connect(p=current_variables['p'][0]*ratio_L5_L2_L3_e2e)
    S_L5_L2_L3_e2i.connect(p=current_variables['p'][1]*ratio_L5_L2_L3_e2i)
    S_L5_L2_L3_i2e.connect(p=current_variables['p'][2]*ratio_L5_L2_L3_i2e)
    S_L5_L2_L3_i2i.connect(p=current_variables['p'][3]*ratio_L5_L2_L3_i2i)
    synapses.append(S_L5_L2_L3_e2e)
    synapses.append(S_L5_L2_L3_e2i)
    synapses.append(S_L5_L2_L3_i2e)
    synapses.append(S_L5_L2_L3_i2i)


    # Since we want the total number of connections from the L6 to the L4 layer to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_L6_L4_e2e_no_dist = Synapses(G_L6_exc, G_L4_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L6_L4_e2e_dist = Synapses(G_L6_exc, G_L4_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L6_L4_e2e_no_dist.connect(p=current_variables['p'][0])
    testing_S_L6_L4_e2e_dist.connect(p=f'{current_variables['p'][0]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L6_L4_e2e = len(testing_S_L6_L4_e2e_dist.N_outgoing_pre) / len(testing_S_L6_L4_e2e_no_dist.N_outgoing_pre)
    
    testing_S_L6_L4_e2i_no_dist = Synapses(G_L6_exc, G_L4_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L6_L4_e2i_dist = Synapses(G_L6_exc, G_L4_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_L6_L4_e2i_no_dist.connect(p=current_variables['p'][1])
    testing_S_L6_L4_e2i_dist.connect(p=f'{current_variables['p'][1]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L6_L4_e2i = len(testing_S_L6_L4_e2i_dist.N_outgoing_pre) / len(testing_S_L6_L4_e2i_no_dist.N_outgoing_pre)

    testing_S_L6_L4_i2e_no_dist = Synapses(G_L6_inh, G_L4_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L6_L4_i2e_dist = Synapses(G_L6_inh, G_L4_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L6_L4_i2e_no_dist.connect(p=current_variables['p'][2])
    testing_S_L6_L4_i2e_dist.connect(p=f'{current_variables['p'][2]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L6_L4_i2e = len(testing_S_L6_L4_i2e_dist.N_outgoing_pre) / len(testing_S_L6_L4_i2e_no_dist.N_outgoing_pre)

    testing_S_L6_L4_i2i_no_dist = Synapses(G_L6_inh, G_L4_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L6_L4_i2i_dist = Synapses(G_L6_inh, G_L4_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_L6_L4_i2i_no_dist.connect(p=current_variables['p'][3])
    testing_S_L6_L4_i2i_dist.connect(p=f'{current_variables['p'][3]}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_L6_L4_i2i = len(testing_S_L6_L4_i2i_dist.N_outgoing_pre) / len(testing_S_L6_L4_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_L6_L4_e2e = Synapses(G_L6_exc, G_L4_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L6_L4_e2e')
    S_L6_L4_e2i = Synapses(G_L6_exc, G_L4_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_L6_L4_e2i')
    S_L6_L4_i2e = Synapses(G_L6_inh, G_L4_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L6_L4_i2e')
    S_L6_L4_i2i = Synapses(G_L6_inh, G_L4_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_L6_L4_i2i')
    S_L6_L4_e2e.connect(p=current_variables['p'][0]*ratio_L6_L4_e2e)
    S_L6_L4_e2i.connect(p=current_variables['p'][1]*ratio_L6_L4_e2i)
    S_L6_L4_i2e.connect(p=current_variables['p'][2]*ratio_L6_L4_i2e)
    S_L6_L4_i2i.connect(p=current_variables['p'][3]*ratio_L6_L4_i2i)
    synapses.append(S_L6_L4_e2e)
    synapses.append(S_L6_L4_e2i)
    synapses.append(S_L6_L4_i2e)
    synapses.append(S_L6_L4_i2i)


    
    ########################################################      
    ## Connections Between Electrode and Electrode layer ##
    ########################################################
    # Configuring synaptic connections between the excitatory neurons and the LFP electrode based on what the layer of the LFP electrode is.
    if current_variables['LFP_electrode_layer'] == "L1":
        S_lfp = Synapses(G_L1_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                               v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    if current_variables['LFP_electrode_layer'] == "L2_L3":
        S_lfp = Synapses(G_L2_L3_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    if current_variables['LFP_electrode_layer'] == "L4":
        S_lfp = Synapses(G_L4_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    if current_variables['LFP_electrode_layer'] == "L5":
        S_lfp = Synapses(G_L5_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    if current_variables['LFP_electrode_layer'] == "L6":
        S_lfp = Synapses(G_L6_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')

    # Ensuring LFP voltage is updated after neuron groups.
    S_lfp.summed_updaters['v_post'].when = 'after_groups' 

    # Generating the connections between the excitatory neurons and the LFP electrode. Here, all of them will be connected since there is no probability defined.
    S_lfp.connect()

    # Setting the weight for LFP calculation which is scaled by the Euclidean distance measure.
    S_lfp.w = '(29e3 * umetre ** 2)/(4*pi*sigma)/((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)**.5'

    synapses.append(S_lfp)


    ############################################
    ########  Final Network Definition  ########
    ############################################
    net = Network(neuron_groups, synapses, monitors)

    
    return net, synapses, monitors

<br></br><br></br>

<h2 style="font-size: 40px;">Simulation Functions</h2>

In [14]:
# Runs the simulation of the neural network in discrete time fragments which includes a mechanism to alter neuron parameters based on a firing rate threshold, which simulates a treatment effect.
def run_granular_simulation(net, variables, treatment_settings, monitors):
    
    print('#######################')
    print('# Starting Simulation #')
    print('#######################')
    print()

    # Extracting the passed on parameters.
    total_duration = variables['duration']
    input_signal = read_input_signal(variables['input_signal_file'])
    time_fragment, firing_rate_threshold = treatment_settings
    popmon_L1_exc, popmon_L2_L3_exc, popmon_L4_exc, popmon_L5_exc, popmon_L6_exc, popmon_L1_inh, popmon_L2_L3_inh, popmon_L4_inh, popmon_L5_inh, popmon_L6_inh, Mlfp, statemon_L1_exc, statemon_L2_L3_exc, statemon_L4_exc, statemon_L5_exc, statemon_L6_exc, statemon_L1_inh, statemon_L2_L3_inh, statemon_L4_inh, statemon_L5_inh, statemon_L6_inh = monitors
    
    # Setting the potassium equilibrium potentials for both the excitatory and inhibitory neurons.
    Eke = variables['Eke_baseline']
    Eki = variables['Eki_baseline']
    Eke_baseline = variables['Eke_baseline']
    Eki_baseline = variables['Eki_baseline']
    
    print('Treatment Parameters:', 'time sensitivity', time_fragment, 'FR Threshold:', firing_rate_threshold)
    print()

    # Converting the time fragments to milliseconds and determining the number of batches.
    time_fragment_ms = int(time_fragment/ms)
    num_batches = int(total_duration / time_fragment)

    # For every batch, we run the simulation on the network 'net'. Here the 'tqdm()' function is used which creates a progress bar.
    for i in tqdm(range(num_batches), desc="Running Simulation"): 
        net.run(time_fragment)

        # If after running a time fragment, the average firing rate of the excitatory neurons over the last time fragment exceeds the threshold, treatment is initiated.
        # This statement refers to the scenario where epilepsy is detected and treated by adjusting the potassium equilibrium potentials of the excitatory and inhibitory neurons.
        if np.mean(popmon_L4_exc.rate[-time_fragment_ms:]) > firing_rate_threshold:
            Eke = variables['Eke_treatment']
            Eki = variables['Eki_treatment']

<br></br>

In [15]:
# This function manages the overall process of setting up and running multiple instances of a neural network simulation as after setting everything up, the function 'run_granular_simulation()' is called.
# In addition, it also handles the storing of the results.
def run_model_loop(variables):

    # Checking whether lengths of each variable list is equal.
    if not check_dict_lenghts(variables):
        raise ValueError('Lenghts of each variable list has be to equal!')
    
    # For every run provided in the 'run_id' field of the variables dictionary we perform the simulation (which requires some setting up first).
    for i in range(len(variables['run_id'])):
        
        ##########################################
        ########  Basic Simulation Setup  ########
        ##########################################
        # Resetting the state of the simulation environment (to avoid interference from the previous run) and seeting the default simulation time step.
        start_scope()
        defaultclock.dt = 0.001*second   

        # Extracting the list of variables required for the current run.
        current_variables = {key: variables[key][i] for key in variables}
        print(current_variables['noise_exc'])

        # Retrieving the treatment settings.
        treatment_settings = [current_variables['device_sensitivity'], current_variables['firing_rate_threshold']]

        # Creating a folder to store the results.
        run_id = current_variables['run_id']
        os.mkdir(f'./results/{run_id}')
        write_run_settings(current_variables, run_id)

        
        #######################################
        ########  Creating Topologies  ########
        #######################################
        # Creating the topologies of both the excitatory and inhibitory neurons.
        layer_data, layer_geometrics, excitatory_positions, inhibitory_positions = create_complete_neuron_topology_cortex(current_variables['N'], current_variables['layer_densities'], current_variables['layer_volumes'], current_variables['excitatory_ratios'], current_variables['layer_names'])
        
        # Assigning the topologies of the excitatory neurons of the different layers to separate variables.
        topology_L1_exc = excitatory_positions['L1']
        topology_L2_L3_exc = excitatory_positions['L2_L3']
        topology_L4_exc = excitatory_positions['L4']
        topology_L5_exc = excitatory_positions['L5']
        topology_L6_exc = excitatory_positions['L6']

        # Assigning the topologies of the inhibitory neurons of the different layers to separate variables.
        topology_L1_inh = inhibitory_positions['L1']
        topology_L2_L3_inh = inhibitory_positions['L2_L3']
        topology_L4_inh = inhibitory_positions['L4']
        topology_L5_inh = inhibitory_positions['L5']
        topology_L6_inh = inhibitory_positions['L6'] 

        # Calculating the lengths of the excitatory and inhibitory topologies.
        excitatory_topologies_lengths = [len(topology_L1_exc[0]), len(topology_L2_L3_exc[0]), len(topology_L4_exc[0]), len(topology_L5_exc[0]), len(topology_L6_exc[0])]
        inhibitory_topologies_lengths = [len(topology_L1_inh[0]), len(topology_L2_L3_inh[0]), len(topology_L4_inh[0]), len(topology_L5_inh[0]), len(topology_L6_inh[0])]

        # Adding all the topologies of the neurons to a list.
        topologies = [topology_L1_exc, topology_L2_L3_exc, topology_L4_exc, topology_L5_exc, topology_L6_exc, topology_L1_inh, topology_L2_L3_inh, topology_L4_inh, topology_L5_inh, topology_L6_inh] 
        topology_names = ['L1_exc', 'L2_L3_exc', 'L4_exc', 'L5_exc', 'L6_exc', 'L1_inh', 'L2_L3_inh', 'L4_inh', 'L5_inh', 'L6_inh']

        # Updating the 'current_variables' by adding the 'layer_data' and 'layer_geometrics'.
        current_variables['layer_data'] = layer_data
        current_variables['layer_geometrics'] = layer_geometrics

        # Plotting all the cortex layers/topologies in a single 3D plot.
        plot_all_cortex_topologies(topologies, topology_names, run_id)


        ############################################
        ########  Setting up Stimulus Mask  ########
        ############################################
        # Defining which topologies should be considered for the creation of the stimulus mask according to the stimulus layers.
        topology_exc_stimulus = [[],[],[]]
        for stimulus_layer in current_variables['stimulus_layers']:
            for i, (sublist1, sublist2) in enumerate(zip_longest(topology_exc_stimulus, excitatory_positions[stimulus_layer], fillvalue=[])):
                topology_exc_stimulus[i].extend(sublist2)

        # If the shape of the stimulus mask is set to 'all', then there is no need to check the mask first.
        if current_variables['shape_stimulus_mask'] == "all":
            
            ##########################################
            ########  Creating Stimulus Mask  ########
            ##########################################
            stimulus_mask_exc = create_all_mask(topology_exc_stimulus)
        
        elif current_variables['shape_stimulus_mask'] == "perc_of_all":

            ##########################################
            ########  Creating Stimulus Mask  ########
            ##########################################
            stimulus_mask_exc = create_prec_of_all_mask(topology_exc_stimulus, current_variables['stimulus_mask_all_perc'])
            
        else:
            # Checking whether the stimulus mask with the provided coordinate center and radius can be applied to the list of layers by calling the function 'checking_mask()'.
            coord_of_stimulus, stimulus_radius_or_edge_length = checking_mask(current_variables['shape_stimulus_mask'], current_variables['stimulus_layers'], current_variables['stimulus_center_coordinates'], current_variables['stimulus_radius_or_edge_length'], current_variables['layer_names'], layer_geometrics)
            
            # Setting the overall geometry settings of the stimulus mask which includes both the middle point as well as the radius.
            stimulus_geometry_settings = [coord_of_stimulus, stimulus_radius_or_edge_length]
            
            ##########################################
            ########  Creating Stimulus Mask  ########
            ##########################################
            # Creating the stimulus mask for the defined 'topology_exc_stimulus' with as shape the 'current_variables['shape_stimulus_mask']'.
            if current_variables['shape_stimulus_mask'] == "spherical":
                stimulus_mask_exc = create_spherical_mask(topology_exc_stimulus, stimulus_geometry_settings)
            elif current_variables['shape_stimulus_mask'] == "cubical":
                stimulus_mask_exc = create_cubical_mask(topology_exc_stimulus, stimulus_geometry_settings)


        #############################################
        ########  Setting up Treatment Mask  ########
        #############################################
        # Defining which topologies should be considered for the creation of the treatment mask according to the treatment layers.
        topology_exc_treatment = [[],[],[]]
        topology_inh_treatment = [[],[],[]]
        for treatment_layer in current_variables['treatment_layers']:
            for i, (sublist1, sublist2) in enumerate(zip_longest(topology_exc_treatment, excitatory_positions[treatment_layer], fillvalue=[])):
                topology_exc_treatment[i].extend(sublist2)
            for i, (sublist1, sublist2) in enumerate(zip_longest(topology_inh_treatment, inhibitory_positions[treatment_layer], fillvalue=[])):
                topology_inh_treatment[i].extend(sublist2)

        # If the shape of the treatment mask is set to 'all', then there is no need to check the mask first.
        if current_variables['shape_treatment_mask'] == "all":

            ###########################################
            ########  Creating Treatment Mask  ########
            ###########################################
            treatment_mask_exc = create_all_mask(topology_exc_treatment)
            treatment_mask_inh = create_all_mask(topology_inh_treatment)

        elif current_variables['shape_treatment_mask'] == "perc_of_all":

            ###########################################
            ########  Creating Treatment Mask  ########
            ###########################################
            treatment_mask_exc = create_prec_of_all_mask(topology_exc_treatment, current_variables['treatment_mask_all_perc'])
            treatment_mask_inh = create_prec_of_all_mask(topology_inh_treatment, current_variables['treatment_mask_all_perc'])
            
        else:
            # Checking whether the treatment mask with the provided coordinate center and radius can be applied to the list of layers by calling the function 'checking_treatment_mask()'.
            coord_of_treatment, treatment_radius_or_edge_length = checking_mask(current_variables['shape_treatment_mask'], current_variables['treatment_layers'], current_variables['treatment_center_coordinates'], current_variables['treatment_radius_or_edge_length'], current_variables['layer_names'], layer_geometrics)
            
            # Setting the overall geometry settings of the treatment mask which includes both the middle point as well as the radius.
            treatment_geometry_settings = [coord_of_treatment, treatment_radius_or_edge_length]
            
            ###########################################
            ########  Creating Treatment Mask  ########
            ###########################################
            # Creating the treatment mask for the defined 'topology_exc_treatment' with as shape the 'current_variables['shape_treatment_mask']'.
            if current_variables['shape_treatment_mask'] == "spherical":
                treatment_mask_exc = create_spherical_mask(topology_exc_treatment, treatment_geometry_settings)
                treatment_mask_inh = create_spherical_mask(topology_inh_treatment, treatment_geometry_settings)
            elif current_variables['shape_treatment_mask'] == "cubical":
                treatment_mask_exc = create_cubical_mask(topology_exc_treatment, treatment_geometry_settings)
                treatment_mask_inh = create_cubical_mask(topology_inh_treatment, treatment_geometry_settings)

        treatment_masks = [treatment_mask_exc, treatment_mask_inh]

        # Plotting the stimulus and treatment masks in a 3D plot which also features all hippocampal topologies.
        plotting_masks_cortex(topologies, topology_names, [current_variables['topology_names_stimulus'], current_variables['topology_names_treatment']], [stimulus_mask_exc, treatment_masks], excitatory_topologies_lengths, inhibitory_topologies_lengths, run_id)

    
    
        ########################################
        ########  Initializing Network  ########
        ########################################
        # Instantiating the network and setting up the monitors.
        net, synapses, monitors = prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables)
        popmon_L1_exc, popmon_L2_L3_exc, popmon_L4_exc, popmon_L5_exc, popmon_L6_exc, popmon_L1_inh, popmon_L2_L3_inh, popmon_L4_inh, popmon_L5_inh, popmon_L6_inh, Mlfp, statemon_L1_exc, statemon_L2_L3_exc, statemon_L4_exc, statemon_L5_exc, statemon_L6_exc, statemon_L1_inh, statemon_L2_L3_inh, statemon_L4_inh, statemon_L5_inh, statemon_L6_inh = monitors

        # Running the simulation with the dynamic objects created above.
        run_granular_simulation(net, current_variables, treatment_settings, monitors)


        ###########################################
        ########  Saving Firing Rate Data  ########
        ###########################################
        exc_color = 'blue'  # Color for excitatory neurons
        inh_color = 'orange'   # Color for inhibitory neurons

        # Saving the firing rate data for every single layer.
        np.savetxt(f'./results/{run_id}/firing_rate_L1_exc.txt', popmon_L1_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L2_L3_exc.txt', popmon_L2_L3_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L4_exc.txt', popmon_L4_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L5_exc.txt', popmon_L5_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L6_exc.txt', popmon_L6_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L1_inh.txt', popmon_L1_inh.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L2_L3_inh.txt', popmon_L2_L3_inh.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L4_inh.txt', popmon_L4_inh.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L5_inh.txt', popmon_L5_inh.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_L6_inh.txt', popmon_L6_inh.rate)
        

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the L1 layer.
        plt.plot(popmon_L1_exc.t, popmon_L1_exc.rate, label='L1_exc', color=exc_color)
        plt.plot(popmon_L1_inh.t, popmon_L1_inh.rate, label='L1_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L1_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L1_exc.t, popmon_L1_exc.rate, label='L1_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L1_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L1_inh.t, popmon_L1_inh.rate, label='L1_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L1_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the L2_L3 layer.
        plt.plot(popmon_L2_L3_exc.t, popmon_L2_L3_exc.rate, label='L2_L3_exc', color=exc_color)
        plt.plot(popmon_L2_L3_inh.t, popmon_L2_L3_inh.rate, label='L2_L3_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L2_L3_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L2_L3_exc.t, popmon_L2_L3_exc.rate, label='L2_L3_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L2_L3_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L2_L3_inh.t, popmon_L2_L3_inh.rate, label='L2_L3_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L2_L3_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the L4 layer.
        plt.plot(popmon_L4_exc.t, popmon_L4_exc.rate, label='L4_exc', color=exc_color)
        plt.plot(popmon_L4_inh.t, popmon_L4_inh.rate, label='L4_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L4_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L4_exc.t, popmon_L4_exc.rate, label='L4_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L4_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L4_inh.t, popmon_L4_inh.rate, label='L4_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L4_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the L5 layer.
        plt.plot(popmon_L5_exc.t, popmon_L5_exc.rate, label='L5_exc', color=exc_color)
        plt.plot(popmon_L5_inh.t, popmon_L5_inh.rate, label='L5_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L5_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L5_exc.t, popmon_L5_exc.rate, label='L5_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L5_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L5_inh.t, popmon_L5_inh.rate, label='L5_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L5_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the L6 layer.
        plt.plot(popmon_L6_exc.t, popmon_L6_exc.rate, label='L6_exc', color=exc_color)
        plt.plot(popmon_L6_inh.t, popmon_L6_inh.rate, label='L6_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L6_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L6_exc.t, popmon_L6_exc.rate, label='L6_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L6_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_L6_inh.t, popmon_L6_inh.rate, label='L6_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_L6_inh.png', bbox_inches='tight')
        plt.close()


        #####################################
        ########  Saving Noise Data  ########
        #####################################
        # Plotting and saving the noise for the excitatory and inhibitory neurons of the L1 layer.
        plt.plot(statemon_L1_exc.t, statemon_L1_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_L1_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_L1_inh.t, statemon_L1_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_L1_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting and saving the noise for the excitatory and inhibitory neurons of the L2_L3 layer.
        plt.plot(statemon_L2_L3_exc.t, statemon_L2_L3_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_L2_L3_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_L2_L3_inh.t, statemon_L2_L3_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_L2_L3_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting and saving the noise for the excitatory and inhibitory neurons of the L4 layer.
        plt.plot(statemon_L4_exc.t, statemon_L4_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_L4_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_L4_inh.t, statemon_L4_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_L4_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting and saving the noise for the excitatory and inhibitory neurons of the L5 layer.
        plt.plot(statemon_L5_exc.t, statemon_L5_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_L5_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_L5_inh.t, statemon_L5_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_L5_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting and saving the noise for the excitatory and inhibitory neurons of the L6 layer.
        plt.plot(statemon_L6_exc.t, statemon_L6_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_L6_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_L6_inh.t, statemon_L6_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_L6_inh.png', bbox_inches='tight')
        plt.close()


        ###################################
        ########  Saving LFP Data  ########
        ###################################
        # Plotting and saving the recorded LFP voltage values in millivolts from the 'Mlfp' monitor. 
        np.savetxt(f'./results/{run_id}/voltage_LFP.txt', Mlfp.v[0]/mV)
        plot(Mlfp.t/ms, Mlfp.v[0]/mV)
        plt.savefig(f'./results/{run_id}/voltage_LFP.png', bbox_inches='tight')
        plt.close()

        
        time.sleep(1)

<br></br><br></br>

<h2 style="font-size: 40px;">Simulation Variables</h2>

In [18]:
########################################
########  Simulation Variables  ########
########################################

# Setting the number of times the simulation will be performed.
copy_times = 1

# Defining the volumes, neuron densities, and excitatory to inhibitory ratios of the layers.
layer_volumes = {"L1": 0.12, 
                 "L2_L3": 0.21, 
                 "L4": 0.10, 
                 "L5": 0.26, 
                 "L6": 0.31}
L2_neuron_density = 95000
L3_neuron_density = 140000
L2_volume = 0.07
L3_volume = 0.14
L2_proportion_L2_L3 = L2_volume / (L2_volume + L3_volume)
L3_proportion_L2_L3 = L3_volume / (L2_volume + L3_volume)
layer_densities_dict = {"L1": 70000, 
                        "L2_L3": L2_proportion_L2_L3 * L2_neuron_density + L3_proportion_L2_L3 * L3_neuron_density,
                        "L4": 190000, 
                        "L5": 120000, 
                        "L6": 100000}
layer_excitatory_ratios_dict = {"L1": 0.8, 
                                "L2_L3": 0.8,
                                "L4": 0.8, 
                                "L5": 0.8, 
                                "L6": 0.8}
                            
# Defining the vocabulary of variables.
variables = {

    # Defining basic run variables.
    "run_id": ['Results 1'],
    "duration": [4000*ms]*copy_times,
    "copy_times": [copy_times]*copy_times,

    # Defining the total number of neurons of the model.
    "N": [17000]*copy_times,

    # Defining the bounds used in the random topology model.
    "bounds": [[0.6, 0.6, 0.6]]*copy_times,
    
    # Defining the number of neurons per mm^3 and the excitatory ratio for each of the layers.
    "layer_names": [['L1', 'L2_L3', 'L4', 'L5', 'L6']]*copy_times,
    "layer_volumes": [dict(layer_volumes)]*copy_times,
    "layer_densities": [dict(layer_densities_dict)]*copy_times,
    "excitatory_ratios": [dict(layer_excitatory_ratios_dict)]*copy_times,

    # Defining the potassium equilibrium potential for both the excitatory and inhibitory neurons.
    # - Healthy mode: Eke_baseline = -90mV
    # - Epileptic mode: Eke_baseline = -84mV
    "Eke_baseline": [-84*mV]*copy_times, 
    "Eki_baseline": [-90*mV]*copy_times,

    # Defining the noise affecting the excitatory and inhibitory neurons.
    "noise_exc": [[0.07, 0.075]*nA]*copy_times, # OLD: [0.1045, 0.104]
    "noise_inh": [[0.05, 0.08]*nA]*copy_times,

    # Defining the base probabilities of connections between neurons from different layers:
    # - p_e2e => Probability of an excitatory to excitatory neuron (synapse) connection.
    # - p_e2i => Probability of an excitatory to inhibitory neuron (synapse) connection.
    # - p_i2e => Probability of an inhibitory to excitatory neuron (synapse) connection.
    # - p_i2i => Probability of an inhibitory to inhibitory neuron (synapse) connection.
    # Normal ranges from 0.7-0.75, to activate sprouting increase the normal by 0.5
    # This will increase the average number of excitatory connections by 500.
    "p": [[0.75, 0.35, 0.35, 0.0]]*copy_times, 

    # Defining from which file the stimulus originates.
    "input_signal_file": ['sigmoid-1.0.txt']*copy_times, 

    ## Defining which parameters can be tweaked for the stimulus mask:
    # - stimulus layers: The layers that will feature the stimulus mask.
    # - stimulus center coordinates: The center coordinates of the stimulus mask.
    # - stimulus radius/edge length: The radius/edge length of the stimulus mask.
    # - stimulus shape: The shape of the stimulus mask.
    "stimulus_layers": [['L4']]*copy_times,
    "topology_names_stimulus": [['L4_exc']]*copy_times,
    "stimulus_center_coordinates": [[0.16, 0.8475, 0.16]]*copy_times,
    "stimulus_radius_or_edge_length": [0.06]*copy_times,
    "shape_stimulus_mask": ['all']*copy_times, # 'spherical', 'cubical', 'all', 'perc_of_all'
    "stimulus_mask_all_perc": [50]*copy_times, # used for the 'create_prec_of_all_mask()' function.
    
    ## Defining which parameters are needed for the creation of the treatment mask:
    # - treatment layers: The layers that will feature the treatment mask.
    # - stimulus center coordinates: The center coordinates of the stimulus mask.
    # - stimulus radius/edge length: The radius/edge length of the stimulus mask.
    # - treatment shape: The shape of the treatment mask.
    "treatment_layers": [['L2_L3']]*copy_times,
    "topology_names_treatment": [['L2_L3_exc', 'L2_L3_inh']]*copy_times,
    "treatment_center_coordinates": [[0.16, 1.06, 0.16]]*copy_times,
    "treatment_radius_or_edge_length": [0.14]*copy_times,
    "shape_treatment_mask": ['spherical']*copy_times, # 'spherical', 'cubical', 'all', 'perc_of_all'
    "treatment_mask_all_perc": [50]*copy_times, # used for the 'create_prec_of_all_mask()' function.
    "distance_between_masks": [0]*copy_times, # the distance required between the stimulus and treatment mask.

    ## Defining other parameters that can be tweaked for treatment:
    # - firing rate threshold: Rate at which the treatment should be activated.
    # - device sensitivity: Frequency with which is checked if the firing rate is above the threshold.
    # - Eke treatment: The potassium equilibrium potential for the excitatory neurons once treatment is activated.
    # - Eke treatment: The potassium equilibrium potential for the inhibitory neurons once treatment is activated (unchanged from untreated).
    "firing_rate_threshold": [5*Hz]*copy_times,
    "device_sensitivity": [8*ms]*copy_times,
    "Eke_treatment": [-100*mV]*copy_times,
    "Eki_treatment": [-90*mV]*copy_times,

    # Defining which layer should feature the LFP electrode.
    "LFP_electrode_layer": ['L4']*copy_times
}

<br></br><br></br>

<h2 style="font-size: 40px;">Running the Simulation</h2>

In [ ]:
# Running the simulation with the variables.
run_model_loop(variables)